In [1]:
import os
import sys
root = os.path.dirname(os.getcwd())
sys.path.append(root)

In [2]:

import tensorflow as tf
from resnet18 import Resnet18
from data_preprocessing import *

2023-07-05 22:39:19.050987: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 22:39:19.873621: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:

chunk = 2
train_path = f'data/chunk_{chunk}.pickle'
test_path = 'data/test_set.pickle'

x_train, y_train, data_size = preprocess_dataset(train_path)
x_test, y_test, _ = preprocess_dataset(test_path)

print('datasize: ', data_size, 'x_train shape:', x_train.shape, 'y_train shape:', y_train.shape, 'x_test shape:', x_test.shape, 'y_test shape:', y_test.shape)


datasize:  4166 x_train shape: (4166, 32, 32, 3) y_train shape: (4166, 10) x_test shape: (10000, 32, 32, 3) y_test shape: (10000, 10)


In [4]:

datagen = get_datagen()
datagen.fit(x_train)


In [5]:
epoch = 200
batch_size = 128
decay_steps = epoch * data_size / batch_size
learning_rate = 0.1
lambda_value = 5e-4


In [6]:
model = Resnet18(num_classes= 10)

# Set the learning rate and decay steps
learning_rate_fn = tf.keras.experimental.CosineDecay(learning_rate, decay_steps=decay_steps)

# Create the SGD optimizer with L2 regularization
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate_fn, momentum=0.9)
regularizer = tf.keras.regularizers.l2(lambda_value)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'], 
              loss_weights=None, weighted_metrics=None, run_eagerly=None, 
              steps_per_execution=None)
# Build the model
model.build(input_shape=(None, 32, 32, 3))


# Apply L2 regularization to applicable layers
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.Dense):
        layer.kernel_regularizer = regularizer
    if hasattr(layer, 'bias_regularizer') and layer.use_bias:
        layer.bias_regularizer = regularizer

2023-07-05 22:39:22.846667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-05 22:39:22.872084: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-05 22:39:22.872159: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-05 22:39:22.875077: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-05 22:39:22.875154: I tensorflow/compile

In [7]:
model.summary()

Model: "resnet18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  1728      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 sequential (Sequential)     (None, 32, 32, 64)        148480    
                                                                 
 sequential_2 (Sequential)   (None, 16, 16, 128)       526848    
                                                                 
 sequential_4 (Sequential)   (None, 8, 8, 256)         2102272   
                                                                 
 sequential_6 (Sequential)   (None, 4, 4, 512)         8398848   
                                                          

In [8]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(patience= 8, restore_best_weights=True, monitor="val_acc")
#I did not use cross validation, so the validate performance is not accurate.
STEPS = len(x_train) / 256

history = model.fit(datagen.flow(x_train,y_train,batch_size = 256), steps_per_epoch=STEPS, batch_size = 256, epochs=50, validation_data=(x_test, y_test),callbacks=[es])

Epoch 1/50


2023-07-05 22:40:48.456527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-07-05 22:40:55.180611: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-05 22:41:06.149930: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fcbc4311100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-05 22:41:06.149981: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1060, Compute Capability 6.1
2023-07-05 22:41:06.744032: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


17/16 [===============================] - ETA: -1s - loss: 4.2096 - accuracy: 0.1510

2023-07-05 22:41:58.614384: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.
2023-07-05 22:41:58.800168: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.


16/16 [==============================] - 84s 3s/step - loss: 4.2096 - accuracy: 0.1510 - val_loss: 122761632.0000 - val_accuracy: 0.1000
Epoch 2/50
16/16 [==============================] - 58s 4s/step - loss: 2.8352 - accuracy: 0.1786 - val_loss: 87580312.0000 - val_accuracy: 0.1000
Epoch 3/50
 1/16 [>.............................] - ETA: 54s - loss: 2.2800 - accuracy: 0.2188

KeyboardInterrupt: 